In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# imputation modules
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [2]:
# dataset path!
dataset_path = '../datasets/bleaching/clean_bleaching.csv'

In [3]:
# read dataset
df = pd.read_csv(dataset_path)

In [4]:
# check dataset
df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Country_Name,Distance_to_Shore,Exposure,Turbidity,...,TSA_Maximum,TSA_Mean,TSA_Frequency,TSA_Frequency_Standard_Deviation,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean
0,1,Donner,-14.28,-170.715,Pacific,Eastern Indo-Pacific,United States,105.68,Sheltered,0.0245,...,2.458,-0.752,1.0,2.238,11.4,1.0,0.00,1.428,9.594,0.564
1,2,Donner,-14.33,-170.500,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.0251,...,2.330,-0.850,1.0,1.520,8.0,1.0,0.00,0.960,8.090,0.340
2,2,Donner,-14.33,-170.500,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.0251,...,2.330,-0.850,1.0,1.520,8.0,1.0,0.00,0.960,8.090,0.340
3,2,Donner,-14.33,-170.500,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.0251,...,2.330,-0.850,1.0,1.520,8.0,1.0,1.57,0.960,8.090,0.340
4,2,Donner,-14.33,-170.500,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.0251,...,2.330,-0.850,0.0,1.520,8.0,1.0,0.00,0.960,8.090,0.340


In [14]:
# ways to clean the dataset
# 1. drop rows with missing values

def drop_missing_values(df):
    """
    Drop rows with missing values
    :param df: dataframe
    :return: dataframe
    """
    # drop rows with missing values
    df.dropna(inplace=True)
    # summarize the number of rows and columns in the dataset
    print(df.shape)
    # summarize the number of rows with missing values for each column
    print(df.isnull().sum())
    # write to csv "../datasets/bleaching/trunc_bleaching.csv"
    df.to_csv('../datasets/bleaching/trunc_bleaching.csv', index=False)

    return df


In [15]:
#2. replace values with the most frequent value in the column

def replace_missing_values(df):
    """
    Replace missing values with the most frequent value in the column
    :param df: dataframe
    :return: dataframe
    """
    # fill missing values with the most frequent value in the column
    df.fillna(df.mode().iloc[0], inplace=True)
    # summarize the number of rows and columns in the dataset
    print(df.shape)
    # summarize the number of rows with missing values for each column
    print(df.isnull().sum())
    # write to csv "../datasets/bleaching/trunc_bleaching.csv"
    df.to_csv('../datasets/bleaching/freq_bleaching.csv', index=False)

    return df


In [5]:
#3. replacing missing categorical values with a neutral value and missing numerical values with the mean

def neutralize_missing_values(df: pd.DataFrame):
    """
    Replacing missing categorical values with a neutral value and missing numerical values with infinity.
    Adds a new column for each existing column, indicating whether the value in the corresponding
    original column is NaN or not.
    :param df: dataframe
    :return: dataframe
    """
    # Identify numerical and categorical columns
    num_cols = df.select_dtypes(include=np.number).columns
    cat_cols = df.select_dtypes(include=['object', 'category']).columns

    # Fill missing categorical values with a neutral value
    # df[cat_cols] = df[cat_cols].fillna('missing')

    # Fill missing numerical values with infinity
    # df[num_cols] = df[num_cols].fillna(np.inf)

    # Add a new column for each existing column, indicating whether the value in the corresponding
    # original column is NaN or not
    for col in df.columns:
        df.insert(df.columns.get_loc(col), f'{col}_isna', df[col].apply(lambda x: 0 if pd.isna(x) else 1))

    # Summarize the number of rows and columns in the dataset
    print(df.shape)

    # Summarize the number of rows with missing values for each column
    print(df.isnull().sum())

    # Write to CSV "../datasets/bleaching/neutral_bleaching.csv"
    df.to_csv('../datasets/bleaching/neutral_bleaching.csv', index=False)

    return df




In [43]:
#4. impuation with sklearn

def impute_missing_values(df):
    """
    Impute missing values for numerical and categorical features
    :param df: dataframe
    :return: dataframe
    """
     # Identify numerical and categorical columns
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Impute numerical columns
    num_imputer = IterativeImputer(estimator=RandomForestRegressor(random_state=0), random_state=0)
    num_imputed = num_imputer.fit_transform(df[num_cols])
    df[num_cols] = num_imputed

    # Impute categorical columns
    cat_df = df[cat_cols].apply(lambda x: x.astype('category'))
    cat_enc = cat_df.apply(lambda x: x.cat.codes)
    cat_imputer = IterativeImputer(estimator=RandomForestClassifier(random_state=0), random_state=0)
    cat_imputed = cat_imputer.fit_transform(cat_enc)
    cat_decoded = pd.DataFrame(cat_imputed, columns=cat_cols).apply(lambda x: x.round().astype(int))
    cat_decoded = cat_decoded.apply(lambda x: pd.Categorical.from_codes(x, categories=cat_df[x.name].cat.categories))
    df[cat_cols] = cat_decoded

    # Write the imputed DataFrame to a CSV file
    df.to_csv('../datasets/bleaching/impute_bleaching.csv', index=False)

    return df

In [0]:
# run the functions

# drop missing values
original_df = df.copy()
drop_missing_values(original_df)

In [0]:

# replace missing values
original_df = df.copy()
replace_missing_values(original_df)

In [6]:

# neutralize missing values
original_df = df.copy()
neutralize_missing_values(original_df)

(37447, 102)
Site_ID_isna                    0
Site_ID                         0
Data_Source_isna                0
Data_Source                     0
Latitude_Degrees_isna           0
                             ... 
TSA_DHW_Standard_Deviation    109
TSA_DHWMax_isna                 0
TSA_DHWMax                    109
TSA_DHWMean_isna                0
TSA_DHWMean                   109
Length: 102, dtype: int64


,Site_ID_isna,Site_ID,Data_Source_isna,Data_Source,Latitude_Degrees_isna,Latitude_Degrees,Longitude_Degrees_isna,Longitude_Degrees,Ocean_Name_isna,Ocean_Name,...,TSA_FrequencyMean_isna,TSA_FrequencyMean,TSA_DHW_isna,TSA_DHW,TSA_DHW_Standard_Deviation_isna,TSA_DHW_Standard_Deviation,TSA_DHWMax_isna,TSA_DHWMax,TSA_DHWMean_isna,TSA_DHWMean
0,1,1,1,Donner,1,-14.280000,1,-170.715000,1,Pacific,...,1,1.0,1,0.00,1,1.4280,1,9.5940,1,0.5640
1,1,2,1,Donner,1,-14.330000,1,-170.500000,1,Pacific,...,1,1.0,1,0.00,1,0.9600,1,8.0900,1,0.3400
2,1,2,1,Donner,1,-14.330000,1,-170.500000,1,Pacific,...,1,1.0,1,0.00,1,0.9600,1,8.0900,1,0.3400
3,1,2,1,Donner,1,-14.330000,1,-170.500000,1,Pacific,...,1,1.0,1,1.57,1,0.9600,1,8.0900,1,0.3400
4,1,2,1,Donner,1,-14.330000,1,-170.500000,1,Pacific,...,1,1.0,1,0.00,1,0.9600,1,8.0900,1,0.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37442,1,1000056,1,Nuryana,1,-8.276040,1,115.593200,1,Indian,...,1,1.0,1,0.00,1,1.7700,1,23.6400,1,0.3900
37443,1,1000057,1,Setiawan,1,-8.362170,1,116.088210,1,Pacific,...,1,1.0,1,1.78,1,1.4900,1,9.0000,1,0.6300
37444,1,1000058,1,Setiawan,1,-8.343690,1,116.050590,1,Pacific,...,1,1.0,1,5.47,1,1.2900,1,8.0100,1,0.6500
37445,1,1000059,1,Setiawan,1,-8.362521,1,116.031430,1,Pacific,...,1,1.0,1,5.47,1,1.2900,1,8.0100,1,0.6500


In [44]:

# impute missing values
original_df = df.copy()
impute_missing_values(original_df)


,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Country_Name,Distance_to_Shore,Exposure,Turbidity,...,TSA_Maximum,TSA_Mean,TSA_Frequency,TSA_Frequency_Standard_Deviation,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean
0,1.0,Donner,-14.280000,-170.715000,Pacific,Eastern Indo-Pacific,United States,105.68,Sheltered,0.024500,...,2.458,-0.7520,1.00,2.238,11.40,1.0,0.00,1.4280,9.5940,0.5640
1,2.0,Donner,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.025100,...,2.330,-0.8500,1.00,1.520,8.00,1.0,0.00,0.9600,8.0900,0.3400
2,2.0,Donner,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.025100,...,2.330,-0.8500,1.00,1.520,8.00,1.0,0.00,0.9600,8.0900,0.3400
3,2.0,Donner,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.025100,...,2.330,-0.8500,1.00,1.520,8.00,1.0,1.57,0.9600,8.0900,0.3400
4,2.0,Donner,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,United States,6923.20,Exposed,0.025100,...,2.330,-0.8500,0.00,1.520,8.00,1.0,0.00,0.9600,8.0900,0.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37442,1000056.0,Nuryana,-8.276040,115.593200,Indian,Central Indo-Pacific,Indonesia,4.79,Sheltered,0.044287,...,2.570,-0.9300,0.00,2.230,11.00,1.0,0.00,1.7700,23.6400,0.3900
37443,1000057.0,Setiawan,-8.362170,116.088210,Pacific,Central Indo-Pacific,Indonesia,84.46,Sheltered,0.037200,...,2.760,-0.9500,3.00,2.000,8.00,1.0,1.78,1.4900,9.0000,0.6300
37444,1000058.0,Setiawan,-8.343690,116.050590,Pacific,Central Indo-Pacific,Indonesia,154.96,Sheltered,0.084900,...,2.860,-1.0000,6.00,1.750,8.00,1.0,5.47,1.2900,8.0100,0.6500
37445,1000059.0,Setiawan,-8.362521,116.031430,Pacific,Central Indo-Pacific,Indonesia,280.59,Sometimes,0.044170,...,2.860,-1.0000,6.00,1.750,8.00,1.0,5.47,1.2900,8.0100,0.6500
